In [1]:
import jax

import jax.numpy as np
import jax.scipy as sp

import numpy as onp
from numpy.polynomial.hermite import hermgauss

import vb_lib.structure_model_lib as structure_model_lib
import vb_lib.cavi_lib as cavi_lib
import vb_lib.structure_optimization_lib as s_optim_lib

import paragami

from copy import deepcopy

import time

/accounts/grad/runjing_liu/.conda/envs/bnp_sensitivity_jax/lib/python3.8/site-packages/jax/lib/xla_bridge.py:130: UserWarning: No GPU/TPU found, falling back to CPU.
  warnings.warn('No GPU/TPU found, falling back to CPU.')


# Load data

In [2]:
# data_file = '../../../../../fastStructure/hgdp_data/huang2011_plink_files/' + \
#                 'phased_HGDP+India+Africa_2810SNPs-regions1to36.npz'

data_file = '../../../../../fastStructure/test/testdata.npz'

data = np.load(data_file)
g_obs = np.array(data['g_obs'], dtype = int)

In [3]:
n_obs = g_obs.shape[0]
n_loci = g_obs.shape[1]

print(n_obs)
print(n_loci)

200
500


# Get prior

In [4]:
prior_params_dict, prior_params_paragami = \
    structure_model_lib.get_default_prior_params()

print('prior params: ')
print(prior_params_dict)

prior params: 
{'dp_prior_alpha': DeviceArray([3.], dtype=float64), 'allele_prior_alpha': DeviceArray([1.], dtype=float64), 'allele_prior_beta': DeviceArray([1.], dtype=float64)}


# VB parameters

In [5]:
k_approx = 25

In [6]:
gh_deg = 8
gh_loc, gh_weights = hermgauss(8)

In [7]:
_, vb_params_paragami = \
    structure_model_lib.get_vb_params_paragami_object(n_obs, n_loci, k_approx,
                                                      use_logitnormal_sticks = True)

In [8]:
s_optim_lib.initialize_with_cavi(g_obs, 
                                 vb_params_paragami, 
                                 prior_params_dict, 
                                 gh_loc, gh_weights, 
                                 max_iter=5)

Compiling cavi functions ...
CAVI compile time: 6.48sec

 running CAVI ...
Done. Warning, max iterations reached. 
Elapsed: 4 steps in 1.76 seconds
Stick conversion time: 9.703secs


OrderedDict([('pop_freq_beta_params',
              DeviceArray([[[228.88026858,   9.80768462],
                            [ 96.99842558,   2.93687052],
                            [ 42.63319623,   2.2728181 ],
                            ...,
                            [  1.00043277,   1.00037146],
                            [  1.00031006,   1.00026613],
                            [  1.0013906 ,   1.00119324]],
              
                           [[200.2334416 ,  38.65548463],
                            [ 69.58227946,  29.93137388],
                            [ 36.01004063,   8.72610057],
                            ...,
                            [  1.00043271,   1.0004195 ],
                            [  1.00031001,   1.00030055],
                            [  1.00139027,   1.00134774]],
              
                           [[ 77.40217414, 161.3065884 ],
                            [ 37.63954449,  61.9639784 ],
                            [ 13.40488309,  31.46864

In [13]:
print(vb_params_dict.keys())

odict_keys(['pop_freq_beta_params', 'ind_admix_params'])


In [14]:
vb_params_dict['pop_freq_beta_params'] = vb_params_dict_beta['pop_freq_beta_params']

stick_betas = vb_params_dict_beta['ind_admix_params']['stick_beta']
logitnorm_stick_params_dict = vb_params_dict['ind_admix_params']
logitnorm_stick_params_paragami = vb_params_paragami['ind_admix_params']

In [15]:
t0 = time.time()
stick_params_dict, out = s_optim_lib.convert_beta_sticks_to_logitnormal(stick_betas, 
                                       logitnorm_stick_params_dict,
                                       logitnorm_stick_params_paragami, 
                                       gh_loc, gh_weights)
print('Stick conversion time: {:.3f}secs'.format(time.time() - t0))

Stick conversion time: 26.292secs


In [16]:
vb_params_dict['ind_admix_params'] = stick_params_dict

In [17]:
structure_model_lib.get_kl(g_obs, vb_params_dict, prior_params_dict, 
                           gh_loc = gh_loc, gh_weights = gh_weights)

DeviceArray(3238135.06908368, dtype=float64)

In [23]:
vb_params_dict['pop_freq_beta_params'].shape

(2810, 30, 2)

# Run Newton

In [21]:
paragami.load_folded?

In [18]:
precond_objective = sp_optim_lib.StructurePrecondObjective(g_obs, 
                                                           vb_params_paragami, 
                                                           prior_params_dict, 
                                                           gh_loc, gh_weights)

NameError: name 'sp_optim_lib' is not defined

In [18]:
out = optimize.minimize(optim_objective.f_np,
                        x0 = onp.array(init_vb_free),
                        jac = optim_objective.grad_np,
                        method='L-BFGS-B', 
                        options = {'maxiter': 5})

NameError: name 'optimize' is not defined